In [ ]:
cd D:/food-detection

In [ ]:
pwd

In [ ]:
# Libraries to be used

# for API call
import requests

# for displaying and directory management
import matplotlib.image as img
%matplotlib inline
import numpy as np
from collections import defaultdict
import collections
from shutil import copy
from shutil import copytree, rmtree
import os
import random

# fro Deep Learning Model
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
from tensorflow.keras import models
import cv2

print('Dependencies Imported')

In [ ]:
# checking the correct version of TF 
print(tf.__version__)
print(tf.test.gpu_device_name())

# validating API key
api_url = 'https://api.calorieninjas.com/v1/nutrition?query='
query = 'sushi'
response = requests.get(api_url + query, headers={'X-Api-Key': 'Sa1uVWZLKXJxM5YPaIEfpg==cEn3gyPDIKhGlmor'})
if response.status_code == requests.codes.ok:
    print(response.text)
else:
    print("Error:", response.status_code, response.text)

In [ ]:
os.listdir('food-101/') 

In [ ]:
os.listdir('food-101/images')

In [ ]:
os.listdir('food-101/meta')

In [ ]:
# display 1 random image of each class
rows = 2
cols = 5
fig, ax = plt.subplots(rows, cols, figsize=(25,25))
fig.suptitle("Showing one random image from each class", y=1.05, fontsize=24)
data_dir = "food-101/images/"
foods_sorted = sorted(os.listdir(data_dir))
food_id = 0
for i in range(rows):
  for j in range(cols):
    try:
      food_selected = foods_sorted[food_id] 
      food_id += 1
    except:
      break
    food_selected_images = os.listdir(os.path.join(data_dir,food_selected))
    food_selected_random = np.random.choice(food_selected_images)
    img = plt.imread(os.path.join(data_dir,food_selected, food_selected_random))
    ax[i][j].imshow(img)
    ax[i][j].set_title(food_selected, pad = 10)
    
plt.setp(ax, xticks=[],yticks=[])
plt.tight_layout()

In [ ]:
# preparing dataset to form test & train with the help of META folder provided with the dataset
def prepare_data(filepath, src, dest):
  classes_images = defaultdict(list)
  with open(filepath, 'r') as txt:
      paths = [read.strip() for read in txt.readlines()]
      for p in paths:
        food = p.split('/')
        classes_images[food[0]].append(food[1] + '.jpg')

  for food in classes_images.keys():
    print("\nCopying images into ",food)
    if not os.path.exists(os.path.join(dest,food)):
      os.makedirs(os.path.join(dest,food))
    for i in classes_images[food]:
      copy(os.path.join(src,food,i), os.path.join(dest,food,i))
  print("Copying Done!")

In [ ]:
print("Creating train data...")
prepare_data('food-101/meta/train.txt', 'food-101/images', 'train')

In [ ]:
print("Creating test data...")
prepare_data('food-101/meta/test.txt', 'food-101/images', 'test')

In [ ]:
foods_sorted

In [ ]:
# selected list of 10 classes
food_list = ['donuts',
 'french_fries',
 'fried_rice',
 'hamburger',
 'hot_dog',
 'pizza',
 'samosa',
 'spring_rolls',
 'sushi',
 'waffles']

In [ ]:
# starting a TF session for training the model 
K.clear_session()
n_classes = 10

# size of target image
img_width, img_height = 299, 299

train_data_dir = 'train'
validation_data_dir = 'test'
train_samples = 7500
validation_samples = 2500
batch_size = 16

# data augmentation for train & test
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# import InceptionV3 Model 
inception = InceptionV3(weights='imagenet', include_top=False)
x = inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(10,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

model = Model(inputs=inception.input, outputs=predictions)
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# checkpointing the model
checkpointer = ModelCheckpoint(filepath='best_model_10class.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history_10class.log')

history = model.fit_generator(train_generator,
                    steps_per_epoch = train_samples // batch_size,
                    validation_data=validation_generator,
                    validation_steps=validation_samples // batch_size,
                    epochs=30,
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])

# saving the model, to be used in the web app
model.save('model_trained_10class.hdf5')


In [ ]:
model.summary()

In [ ]:
class_map_10 = train_generator.class_indices
class_map_10

In [ ]:
def plot_accuracy(history,title):
    plt.title(title)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy'], loc='best')
    plt.show()
def plot_loss(history,title):
    plt.title(title)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'validation_loss'], loc='best')
    plt.show()


In [ ]:
plot_accuracy(history,'FOOD101-Inceptionv3')
plot_loss(history,'FOOD101-Inceptionv3')

In [ ]:
# loading the model
K.clear_session()
print("Loading the model..")
model = load_model('best_model_10class.hdf5',compile = False)
print("Done!")

In [ ]:
# prediction function
def predict_class(model, images):
  for img in images:
    img = image.load_img(img, target_size=(299, 299))
    img = image.img_to_array(img)                    
    img = np.expand_dims(img, axis=0)         
    img /= 255.                                      

    pred = model.predict(img)
    index = np.argmax(pred)
    food_list.sort()
    pred_value = food_list[index]
    api_url = 'https://api.calorieninjas.com/v1/nutrition?query='
    query = pred_value.replace("_"," ")
    response = requests.get(api_url + query, headers={'X-Api-Key': 'Sa1uVWZLKXJxM5YPaIEfpg==cEn3gyPDIKhGlmor'})
    if response.status_code == requests.codes.ok:
        print(response.text)
    else:
        print("Error:", response.status_code, response.text)

In [ ]:
# trying on test images
images = []
images.append('img/burger.jpg')
images.append('img/samosa.jpg')
predict_class(model, images)